In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as ms
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv("./RNA_seq.csv")
dataT=np.array(data)
data=dataT.T
co=data[0]
data1=np.delete(data,0,axis=0)
data=data1
datadf= pd.DataFrame(data=data[0:,0:],columns=co)
datadf.head()
data=datadf
data= data.replace("NOTLC",value=0)
data= data.replace("LC",value=1)
X=data.drop(['Group'],axis=1)
y=data['Group']

In [6]:
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)
X=pd.DataFrame(data=X,columns=column)

In [7]:
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("cross-validate across the entire data set")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    a=accuracy_score(ytrain,y_pred_cross)
    p=precision_score(ytrain, y_pred_cross)
    r=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    wf1=f1_score(ytrain, y_pred_cross, average='weighted')
    #auc=roc_auc_score(ytrain,estimator.predict_proba(Xtrain)[:,1])
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',a)
    print("the precision score of the model is : ", p)
    print("the recall score of the model is :", r)
    print('the f1_score of the model  is :',f1)
    print('the weighted_f1 of the model is :',wf1)
    print('the classification_report is :\n',classification_report(ytrain, y_pred_cross,digits=4))
    #print('the auc is :',auc)
    return a,p,r,f1

In [8]:
from sklearn.feature_selection import SequentialFeatureSelector
def searchNFeatures(estimator,nEnd = 11):
    n_feature_range = range(3,nEnd+1)
    accmax2=0
    premax2=0
    recmax2=0
    f1max2=0
    featuremax2=0
    feature2=[]
    for n in n_feature_range:
        print("N features:", n)
        sel_seq = SequentialFeatureSelector(estimator=estimator, n_features_to_select=n)
        temp=sel_seq.fit(X, y)
        sel_seq_mask= X.columns[temp.get_support()]
        print('features:',sel_seq_mask)
        X_new=sel_seq.fit_transform(X,y)
        acc2,pre2,rec2,f12=crosspredict(estimator,X_new,y,10)
        if f12>f1max2:
            f1max2=f12
            featuremax2=n
            accmax2=acc2
            premax2=pre2
            recmax2=rec2
            feature2=sel_seq_mask
    
    print("整个数据集上交叉验证：")
    print('feature :',feature2)
    print('the best feature number is:',featuremax2)
    print('the best accuracy is:',accmax2)
    print('the best precision is:',premax2)
    print('the best recall is:',recmax2)
    print('the best f1_score is:',f1max2)
    print("\n")

In [ ]:
searchNFeatures(SVC(),10)

'ENSG00000002726.21','ENSG00000039068.19','ENSG00000120279.6','ENSG00000158578.21','ENSG00000182685.7','ENSG00000204305.14'

In [ ]:
searchNFeatures(RandomForestClassifier(),10)

'ENSG00000106089.12', 'ENSG00000204305.14', 'ENSG00000223982.3',
       'ENSG00000229693.2'

In [ ]:
searchNFeatures(KNeighborsClassifier(),10)

'ENSG00000012983.11','ENSG00000106089.12','ENSG00000158578.21','ENSG00000204305.14','ENSG00000214796.8', 'ENSG00000277878.1'

In [ ]:
searchNFeatures(DecisionTreeClassifier(),10)

'ENSG00000003147.19','ENSG00000004455.17','ENSG00000006327.14','ENSG00000018510.17','ENSG00000067064.11','ENSG00000154813.10','ENSG00000204305.14','ENSG00000234425.1','ENSG00000262772.2','ENSG00000272477.1'

In [ ]:
searchNFeatures(AdaBoostClassifier(),10)

'ENSG00000000005.6', 'ENSG00000002933.9','ENSG00000019144.19', 'ENSG00000070366.14','ENSG00000091262.16','ENSG00000106089.12','ENSG00000204305.14','ENSG00000233117.4'

In [ ]:
searchNFeatures(GaussianNB(),10)

'ENSG00000011052.21', 'ENSG00000052841.15',
       'ENSG00000062194.16', 'ENSG00000068001.14', 'ENSG00000102871.16',
       'ENSG00000124532.15', 'ENSG00000204305.14', 'ENSG00000227066.2'

In [ ]:
searchNFeatures(BernoulliNB(),10)

'ENSG00000000003.15','ENSG00000074527.13','ENSG00000170323.9','ENSG00000224215.1','ENSG00000268221.6', 'ENSG00000272953.1'

In [ ]:
searchNFeatures(LogisticRegression(),10)

'ENSG00000102547.19','ENSG00000135604.10','ENSG00000159352.16','ENSG00000224215.1','ENSG00000234481.2','ENSG00000252275.1','ENSG00000271555.1'

In [ ]:
searchNFeatures(GradientBoostingClassifier(),10)

'ENSG00000000005.6','ENSG00000000971.16', 'ENSG00000001631.16','ENSG00000002587.10', 'ENSG00000004779.10','ENSG00000106089.12', 'ENSG00000144130.11', 'ENSG00000204305.14'

In [ ]:
searchNFeatures(XGBClassifier(),10)

'ENSG00000106089.12','ENSG00000138346.15','ENSG00000204305.14'

In [ ]:
searchNFeatures(lgb.LGBMClassifier(),10)

'ENSG00000002834.18','ENSG00000106089.12', 'ENSG00000170989.10','ENSG00000204305.14'